**Extracting answers from claim notes**

**Author:** Matthias Quinn

**Date:** March 17th, 2022

**Project:** AVT

In [3]:
import numpy as np
import pandas as pd
import re

In [4]:
claims = pd.read_excel("Quinn_AVT Workspace.xlsx", sheet_name="AVT Claims", index_col=None)

In [5]:
claims.head()

,WorkerFullName,DirectManagerName,AVTGroup,ReportDate,ClaimYear,ClaimNumber,FullClaim,ClaimNote,REP_CD,CLRU_REP_CD,ClaimReportByCategory,VehicleYear,VehicleMake,VehicleModel,Type
0,"Williams,Shaquan W.","Allen,Princess S",B,2022-02-25,22,2215002,22-2215002,IMMEDIATE CONCERNS: ...,A162742,A156222,INSURED,20,HYUNDAI,SANTA FE,INTERSECTION
1,"Rivera,Shalanda N","Loncaric,Christopher Anton",A,2022-02-28,22,2215835,22-2215835,AVT FNOL QUESTIONS ...,A162321,A164246,INSURED,21,MAZDA,CX-30,INTERSECTION
2,"Duff,Lacreasha","Faussett,Jason Curtis",A,2022-03-01,22,2216037,22-2216037,IMMEDIATE CONCERNS: ...,A144314,A161370,INSURED,20,FORD,ESCAPE,MERGING
3,"Pounds,Toni","Allen,Princess S",B,2022-03-02,22,2216479,22-2216479,AVT FNOL QUESTIONS ...,A109573,A160374,INSURED,20,CHEVROLET,MALIBU,LEFT OF CENTER
4,"Borally,Estacia Emily","Loncaric,Christopher Anton",A,2022-03-03,22,2216846,22-2216846,IMMEDIATE CONCERNS: ...,A157192,A148719,INSURED,21,TOYOTA,4RUNNER,REAREND


**Retrieve just the claim notes:**

In [6]:
claimNotes = claims["ClaimNote"]

claimNotes.head()

0    IMMEDIATE CONCERNS:                           ...
1    AVT FNOL QUESTIONS                            ...
2    IMMEDIATE CONCERNS:                           ...
3    AVT FNOL QUESTIONS                            ...
4    IMMEDIATE CONCERNS:                           ...
Name: ClaimNote, dtype: object

In [25]:
sampleB = "IMMEDIATE CONCERNS:                                                            AVT FNOL QUESTIONS                                                             -                                                                               DOES YOUR VEHICLE HAVE ANY TECHNOLOGY THAT ALLOWS IT TO STEER, BRAKE AND      ACCELERATE ON ITS OWN? NO IF YES, DID YOU TURN ON ANY OF THAT TECHNOLOGY       BEFORE THE LOSS? YES OR NO NO                                                  -                                                                              FOR JUMPSTART TEST ONLY"
print(sampleB)

IMMEDIATE CONCERNS:                                                            AVT FNOL QUESTIONS                                                             -                                                                               DOES YOUR VEHICLE HAVE ANY TECHNOLOGY THAT ALLOWS IT TO STEER, BRAKE AND      ACCELERATE ON ITS OWN? NO IF YES, DID YOU TURN ON ANY OF THAT TECHNOLOGY       BEFORE THE LOSS? YES OR NO NO                                                  -                                                                              FOR JUMPSTART TEST ONLY


In [50]:
sampleA = "AVT FNOL QUESTIONS                                                             -                                                                               BEFORE THE LOSS, DID YOU TURN ON ANY TECHNOLOGY THAT ALLOWS YOUR VEHICLE TO   STEER, BRAKE AND ACCELERATE ON IT'S OWN? YES OR NONO                           -                                                                              FOR JUMPSTART TEST ONLY"
print(sampleA)

AVT FNOL QUESTIONS                                                             -                                                                               BEFORE THE LOSS, DID YOU TURN ON ANY TECHNOLOGY THAT ALLOWS YOUR VEHICLE TO   STEER, BRAKE AND ACCELERATE ON IT'S OWN? YES OR NONO                           -                                                                              FOR JUMPSTART TEST ONLY


In [51]:
def cleanNote(claimNote):
    upperNote = str(claimNote).upper()
    trimNote = " ".join(upperNote.split())
    spaceNote = trimNote.replace("  ", " ")
    # spaceNote = spaceNote.replace("NONO", "NO NO")
    # spaceNote = spaceNote.replace("YESYES", "YES YES")
    spaceNote = re.sub(',', " ", spaceNote)
    btwnNote = re.search('AVT FNOL QUESTIONS(.*?)ONLY', spaceNote).group(1)
    removeWords = ["TECHNOLOGY", "NOTE", "FNOL"]
    splitNote = [i for i in btwnNote.split(" ") if i not in removeWords]
    keepNote = " ".join(splitNote)

    return keepNote

cleanB = cleanNote(sampleB)
cleanA = cleanNote(sampleA)

cleanA

" - BEFORE THE LOSS  DID YOU TURN ON ANY THAT ALLOWS YOUR VEHICLE TO STEER  BRAKE AND ACCELERATE ON IT'S OWN? YES OR NONO - FOR JUMPSTART TEST "

In [58]:
def groupA(cleanNote: str, group: str):
    "Extracts the answer to the question from the claim note for only Group A"
    Nos = cleanNote.count("NO")
    Yeses = cleanNote.count("YES")

    print({"Nos": Nos, "Yeses": Yeses})

    if group == "A" and (Nos % 2 == 0) and "YES OR NO" in cleanNote:
        return "NO"
    
    elif group == "A" and (Nos % 2 == 0) and "YES OR NO" not in cleanNote:
        return "NO"
    
    else:
        return "YES"

groupA(cleanA, "A")

{'Nos': 2, 'Yeses': 1}


'NO'